# Рубежный контроль №2. Милевич Артём ИУ5-23М
### Тема: Методы обработки текстов
Необходимо решить задачу классификации текстов на основе любого выбранного Вами датасета (SMSSpamCollection). Классификация может быть бинарной или многоклассовой. Целевой признак из выбранного Вами датасета может иметь любой физический смысл, примером является задача анализа тональности текста.

Необходимо сформировать два варианта векторизации признаков - на основе CountVectorizer и на основе TfidfVectorizer.

В качестве классификаторов необходимо использовать два классификатора: LinearSVC, LogisticRegression.

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

### 1. Загрузка данных

In [7]:
df = pd.read_csv('../SMSSpamCollection', sep='\t', header=None, names=['label', 'text'])

Преобразуем метки в бинарный формат: ham=0, spam=1

In [8]:
df['label_num'] = df['label'].map({'ham':0, 'spam':1})

### 2. Разделение на обучающую и тестовую выборки

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['label_num'], test_size=0.2, random_state=42, stratify=df['label_num']
)

### 3. Определяем векторизаторы

In [10]:
vectorizers = {
    'CountVectorizer': CountVectorizer(),
    'TfidfVectorizer': TfidfVectorizer()
}

### 4. Обучение и оценка моделей

In [11]:
results = {}

for vect_name, vect in vectorizers.items():
    # Векторизация текста
    X_train_vect = vect.fit_transform(X_train)
    X_test_vect = vect.transform(X_test)
    
    # LinearSVC
    svc = LinearSVC(random_state=42)
    svc.fit(X_train_vect, y_train)
    y_pred_svc = svc.predict(X_test_vect)
    acc_svc = accuracy_score(y_test, y_pred_svc)
    report_svc = classification_report(y_test, y_pred_svc, target_names=['ham', 'spam'])
    results[f'{vect_name} + LinearSVC'] = (acc_svc, report_svc)
    
    # LogisticRegression
    lr = LogisticRegression(max_iter=1000, random_state=42)
    lr.fit(X_train_vect, y_train)
    y_pred_lr = lr.predict(X_test_vect)
    acc_lr = accuracy_score(y_test, y_pred_lr)
    report_lr = classification_report(y_test, y_pred_lr, target_names=['ham', 'spam'])
    results[f'{vect_name} + LogisticRegression'] = (acc_lr, report_lr)

### 5. Вывод результатов

In [12]:
for model_name, (acc, report) in results.items():
    print(f'=== Модель: {model_name} ===')
    print(f'Точность (accuracy): {acc:.4f}')
    print('Отчёт по классификации:')
    print(report)
    print('\n' + '-'*60 + '\n')

=== Модель: CountVectorizer + LinearSVC ===
Точность (accuracy): 0.9830
Отчёт по классификации:
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       966
        spam       1.00      0.87      0.93       149

    accuracy                           0.98      1115
   macro avg       0.99      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115


------------------------------------------------------------

=== Модель: CountVectorizer + LogisticRegression ===
Точность (accuracy): 0.9812
Отчёт по классификации:
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       966
        spam       1.00      0.86      0.92       149

    accuracy                           0.98      1115
   macro avg       0.99      0.93      0.96      1115
weighted avg       0.98      0.98      0.98      1115


------------------------------------------------------------

=== Модель: TfidfVe

### Выводы по результатам классификации SMS Spam Collection
1. Высокая точность всех моделей
    - Все четыре модели продемонстрировали высокую точность (accuracy) — от 97.3% до 98.5%, что говорит о том, что выбранные методы векторизации и классификации хорошо справляются с задачей бинарной классификации спам/не спам.

2. Сравнение векторизаций
    - Модели с TfidfVectorizer показали чуть более высокую точность и лучшие метрики recall и f1-score для класса спам по сравнению с CountVectorizer, особенно заметно у LinearSVC (accuracy 98.48% против 98.30%).
    - Это объясняется тем, что TF-IDF учитывает важность слов в контексте всего корпуса, что помогает лучше выделять характерные признаки спама.

3. Сравнение классификаторов
    - LinearSVC в сочетании с TfidfVectorizer дал наилучший результат по recall (0.90) и f1-score (0.94) для класса спам, что важно, так как пропуск спам-сообщений нежелателен.
    - LogisticRegression показал немного меньшие показатели по recall для спама, особенно с TfidfVectorizer (0.80), что может означать больше пропущенных спам-сообщений.

4. Баланс precision и recall
    - Все модели демонстрируют очень высокий precision для класса спам (около 0.99–1.00), то есть почти все предсказанные спам-сообщения действительно являются спамом.
    - Recall для спама чуть ниже, что указывает на некоторый процент пропущенных спам-сообщений, но в целом результаты остаются очень хорошими.

5. Рекомендации
    - Для задач, где важно минимизировать пропуск спама, рекомендуется использовать LinearSVC с TfidfVectorizer.
    - Если важна простота и скорость, CountVectorizer с LinearSVC или LogisticRegression также дают достойные результаты.

6. Общее заключение
    - Данные результаты подтверждают, что классические методы векторизации и линейные модели остаются эффективными для текстовой классификации на небольших и средних датасетах. Векторизация с помощью TF-IDF улучшает качество распознавания за счёт более информативного представления текста.